In [2]:
# Import the libraries
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import datetime as dt
import pandas_datareader.data as web
import yfinance as yf

# overide default yfinance download method
yf.pdr_override()
# set backend of pandas to plotly
pd.options.plotting.backend = "plotly"

In [3]:
class OptionsData:

    def __init__(self, ticker: str, expirey: dt.date, option_type: str):
        self.ticker = ticker
        self.expirey = expirey
        self.option_type = option_type
        self.options_data = self.get_options_data()

    
    def get_options_data(self) -> pd.DataFrame:
        """ 
        This function will return a pandas dataframe of the options data for a given ticker, 
        expirey date, and option type. It will return the closest expirey date to the given date
        
        Parameters
        ----------
        ticker : str
            The ticker of the stock you want to get the options data for
        date : datetime.date
            The date you want to get the options data for
        type : str
            The type of option you want to get the data for. Must be either 'call' or 'put'
        
        Returns
        -------
        data : pd.DataFrame
            A pandas dataframe of the options data for the given ticker, expirey date, and option type
        """
        try:    
            options = web.YahooOptions(self.ticker)
            options.headers = {'User-Agent': 'Firefox'}
        except:
            print("Error: Could not get options data for ticker: ", self.ticker)
            return None
    
        # Get closest expirey date to date and reset self.expirey to
        expirey_new = min(options._get_expiry_dates(), key=lambda x: abs(x - self.expirey))
        if self.expirey != expirey_new:
            print("#---------------------#")
            print("Closest expirey date to", self.expirey, "is", expirey_new)
            print("Using expirey date:", expirey_new)
            print("#---------------------#")
            self.expirey = expirey_new
        

        # Get the call data
        if type == "call":
            data = options.get_call_data(expiry=self.expirey)
        elif type == "put":
            data = options.get_put_data(expiry=self.expirey)
        else:
            raise ValueError("option_type must be either 'call' or 'put'")

        # The JSON data is the actual useful data that we want
        data.reset_index(inplace=True)
        data.set_index("Strike", inplace=True)
        data = data.JSON.apply(pd.Series)

        #Drop strike column since it is now the index
        data.drop("strike", axis=1, inplace=True)

        return data

    def get_underlying_data(self, start: dt.date, end: dt.date, interval: str = "1d") -> pd.DataFrame:
        """ 
        This function will return a pandas dataframe of the underlying data for a given ticker, 
        start date, end date, and interval
        
        Parameters
        ----------
        ticker : str
            The ticker of the stock you want to get the options data for
        start : datetime.date
            The start date you want to get the underlying data for
        end : datetime.date
            The end date you want to get the underlying data for
        interval : str
            The interval you want to get the underlying data for. Must be either '1d', '1wk', '1mo'
        
        Returns
        -------
        data : pd.DataFrame
            A pandas dataframe of the underlying data for the given ticker, start date, end date, and interval
        """
        try:
            print("#---------------------#")
            print("Getting underlying data for ticker: ", self.ticker)
            print("#---------------------#")
            data = web.get_data_yahoo(self.ticker, start, end, interval=interval)
        except:
            print("Error: Could not get underlying data for ticker: ", self.ticker)
            return None
        
        return data
    

In [50]:
ticker = "GS"
date = dt.date.today() + dt.timedelta(days=60)
type = "put"

data = OptionsData(ticker, date, type)
options_df = data.options_data
stock_df = data.get_underlying_data(dt.date(2021, 12, 31), dt.date.today(),  "1d")

#---------------------#
Closest expirey date to 2023-06-03 is 2023-06-16
Using expirey date: 2023-06-16
#---------------------#
#---------------------#
Getting underlying data for ticker:  GS
#---------------------#
[*********************100%***********************]  1 of 1 completed


In [51]:
# Plot the last price * volume of the underlying vs the strike price
colors = ['green' if val else 'red' for val in options_df.inTheMoney]

fig = go.Figure()
fig.add_trace(go.Scatter(x=options_df.index, y=options_df.lastPrice * options_df.volume, mode='markers', marker_color=colors, name="Last Price * Volume"))

fig.update_layout(
    xaxis_title="Strike Price",
    yaxis_title="Last Price * Volume",
    font=dict(
        size=18,
    )

)

# scale y axis to be log
fig.update_yaxes(type="log")

fig.show()

In [33]:
# Plot strike vs bid where data point is coloured by in the money
# in the money true = green, false = red

colors = ['green' if val else 'red' for val in options_df.inTheMoney]

fig = go.Figure()
fig.add_trace(go.Scatter(x=options_df.index, y=options_df.lastPrice, mode='markers', marker_color=colors))
fig.add_trace(go.Scatter(x=stock_df.index, y=stock_df.Close, mode='lines', name='Close'))
fig.update_layout(title="Strike vs Bid", xaxis_title="Strike", yaxis_title="Bid")

fig.show()

In [290]:
# plot implied volatility vs strike
fig = go.Figure()
fig.add_trace(go.Scatter(x=options_df.index, y=options_df.impliedVolatility, mode='lines'))

fig.update_layout(title="Strike vs Implied Volatility", xaxis_title="Strike", yaxis_title="Implied Volatility")
fig.show()

In [291]:
stock_df

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2021-12-31,385.000000,387.000000,380.839996,382.549988,370.066864,1601300
2022-01-03,389.000000,399.299988,387.339996,395.329987,382.429810,3334300
2022-01-04,402.230011,410.279999,401.179993,407.480011,394.183380,3916400
2022-01-05,409.799988,412.660004,397.890015,398.630005,385.622162,3469600
2022-01-06,401.000000,401.200012,386.429993,396.929993,383.977600,2960900
...,...,...,...,...,...,...
2023-03-20,306.779999,312.200012,306.260010,309.529999,309.529999,3963700
2023-03-21,317.619995,320.320007,315.350006,317.279999,317.279999,3242400
2023-03-22,317.279999,322.220001,313.489990,313.670013,313.670013,2505900


In [338]:
# Define the ticker symbol for the repo rate
ticker = 'DPRIME'
start_date = '2001-01-01'
end_date = dt.date.today().strftime("%Y-%m-%d")

cash_rate= web.FredReader(symbols=['DFF', 'DFEDTAR', 'DFEDTARL', 'DFEDTARU'], start=start_date, end=end_date).read()

In [339]:
# Plot the repo rate as a line chart
fig = go.Figure()
fig.add_trace(go.Scatter(x=cash_rate.index, y=cash_rate.DFF, mode='lines', name='Cash Rate'))
fig.add_trace(go.Scatter(x=cash_rate.index, y=cash_rate.DFEDTAR, mode='lines', name='Target Rate'))
fig.add_trace(go.Scatter(x=cash_rate.index, y=cash_rate.DFEDTARL, mode='lines', name='Target Rate Lower Bound'))
fig.add_trace(go.Scatter(x=cash_rate.index, y=cash_rate.DFEDTARU, mode='lines', name='Target Rate Upper Bound'))
fig.update_layout(title="Cash Rate", xaxis_title="Date", yaxis_title="Rate")
fig.show()

In [340]:
ticker = "BKFTTLA641N"
failures = web.FredReader(symbols=ticker, start=start_date, end=end_date).read()



In [342]:
# Plot failures as a line chart
fig = go.Figure()
fig.add_trace(go.Scatter(x=failures.index, y=failures.BKFTTLA641N, mode='lines', name='Failures'))
fig.update_layout(title="Failures", xaxis_title="Date", yaxis_title="Failures")
fig.show()

In [19]:


# Specify the ticker symbol and the start and end dates
ticker = "AAPL"
start_date = "2023-04-06"
end_date = "2023-04-10"

# Get the options data using the history() method
options_data = yf.Ticker(ticker).option_chain(date=start_date)

# Filter the options data to only include data for the end date
# options_data = options_data.loc[end_date]

# Print the options data
print(options_data)


Options(calls=         contractSymbol             lastTradeDate  strike  lastPrice   bid  \
0   AAPL230406C00050000 2023-04-03 19:55:00+00:00    50.0     116.05   0.0   
1   AAPL230406C00055000 2023-04-03 19:40:57+00:00    55.0     110.90   0.0   
2   AAPL230406C00060000 2023-04-03 18:53:22+00:00    60.0     105.65   0.0   
3   AAPL230406C00065000 2023-03-31 17:06:11+00:00    65.0      98.65   0.0   
4   AAPL230406C00070000 2023-03-31 19:59:56+00:00    70.0      94.80   0.0   
5   AAPL230406C00075000 2023-03-27 18:50:23+00:00    75.0      84.00   0.0   
6   AAPL230406C00080000 2023-03-09 17:02:23+00:00    80.0      74.15   0.0   
7   AAPL230406C00085000 2023-03-29 14:06:13+00:00    85.0      75.25   0.0   
8   AAPL230406C00090000 2023-03-02 14:36:53+00:00    90.0      55.00  73.6   
9   AAPL230406C00095000 2023-03-31 19:55:00+00:00    95.0      69.85   0.0   
10  AAPL230406C00100000 2023-03-31 19:54:40+00:00   100.0      64.70   0.0   
11  AAPL230406C00105000 2023-03-31 19:54:40+00:00 

In [20]:
options_df

,contractSymbol,currency,lastPrice,change,percentChange,volume,openInterest,bid,ask,contractSize,expiration,lastTradeDate,impliedVolatility,inTheMoney
Strike,,,,,,,,,,,,,,
200.0,GS230406P00200000,USD,0.02,0.0,0.0,5.0,0,0.00,0.00,REGULAR,1680739200,1679925846,0.500005,False
210.0,GS230406P00210000,USD,0.05,0.0,0.0,5.0,0,0.00,0.00,REGULAR,1680739200,1679666920,0.500005,False
215.0,GS230406P00215000,USD,0.02,0.0,0.0,65.0,0,0.00,0.00,REGULAR,1680739200,1679683558,0.500005,False
220.0,GS230406P00220000,USD,0.04,0.0,0.0,13.0,0,0.00,0.00,REGULAR,1680739200,1679683569,0.500005,False
225.0,GS230406P00225000,USD,0.03,0.0,0.0,NaN,0,0.00,0.00,REGULAR,1680739200,1679512082,0.500005,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375.0,GS230406P00375000,USD,61.50,0.0,0.0,1.0,0,0.00,0.00,REGULAR,1680739200,1678984706,0.000010,True
380.0,GS230406P00380000,USD,60.44,0.0,0.0,2.0,0,0.00,0.00,REGULAR,1680739200,1678717549,0.000010,True
390.0,GS230406P00390000,USD,39.53,0.0,0.0,NaN,0,61.90,64.30,REGULAR,1680739200,1677682484,1.196293,True
